In [1]:
from urllib.request import urlopen,urlparse
from bs4 import BeautifulSoup
import re
from urllib.error import URLError,HTTPError

In [ ]:

def extractdata(url):
    try:
        html = urlopen(url)
        bs = BeautifulSoup(html, 'html.parser')
        for child in bs.find('table',{'id':'giftList'}).tr.next_siblings:
            print(child)
    except HTTPError as e:
        print(f"HTTPError occurred: {e}")
    except URLError as e:
        print(f"URLError occurred: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

extractdata("https://www.pythonscraping.com/pages/page3.html")


In [ ]:
import re
def extractdata1(url):
    try:
        html=urlopen(url)
        bs=BeautifulSoup(html,'html.parser')
        print(bs.find('img', {'src':'../img/gifts/img1.jpg'}).parent.previous_sibling.get_text())
    except URLError:
        print("couldnot acess the url")
    except HTTPError as e:
        print(f"the error is:{e}")

extractdata1("https://www.pythonscraping.com/pages/page3.html")       

In [ ]:
url="https://www.pythonscraping.com/pages/page3.html"
html=urlopen(url)
bs=BeautifulSoup(html,'html.parser')
images=bs.find_all('img',{'src':re.compile(r"^.+\/img\/.+\/img.*\.jpg$")})
for image in images:
    print(image['src'])

In [ ]:
first_part=bs.find('div',{'id':'footer'})
print(first_part.get_text(strip=True))

In [ ]:

url='http://en.wikipedia.org/wiki/Kevin_Bacon'
html=urlopen(url)
bs=BeautifulSoup(html,'html.parser')
for link in bs.find('div',{'id':'bodyContent'}).find_all('a',href=re.compile(r"^(/wiki/)((?!:).)*$")):
    if 'href' in link.attrs:
        print(link.attrs['href'])

In [ ]:
import datetime
import random
random.seed(datetime.datetime.now().timestamp())
def getLinks(articleUrl):
    url=f"http://en.wikipedia.org{articleUrl}"
    html=urlopen(url)
    bs=BeautifulSoup(html,'html.parser')
    return bs.find('div',{'id':'bodyContent'}).find_all('a',href=re.compile(r"^(/wiki/)((?!:).)*$"))

links=getLinks('/wiki/Kevin_Bacon')
count=0
while count<=30:
    newArticle=links[random.randint(0,len(links)-1)].attrs['href']
    links=getLinks(newArticle)
    count+=1
print(newArticle)
url=f"http://en.wikipedia.org/wiki/Bruno_Urli%C4%87"
    

In [ ]:
pages=set()
def getLinks(pageUrl):
    html=urlopen(f'https://en.wikipedia.org{pageUrl}')
    bs=BeautifulSoup(html,'html.parser')
    try:
        print(bs.h1.get_text())

    except AttributeError:
        print("this page is missing something!!Continuing")
    
    for link in bs.find_all('a',href=re.compile("^(/wiki/)")):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                newPage=link.attrs['href']
                print('-'*20)
                print(newPage)
                pages.add(newPage)
                getLinks(newPage)                
articleUrl='/wiki/Emma_Watson'
getLinks(articleUrl)

In [ ]:
def getInternalLinks(bs,url):
    netloc=urlparse(url).netloc
    scheme=urlparse(url).scheme
    internalLinks=set()
    for link in bs.find_all('a'):
        if not links.attrs.get('href'):
            continue
        parsed=urlparse(link.attrs['href'])
        if parsed.netloc=='':
            l=f'{scheme}://{netloc}/{link.attrs["href"].strip("/")}'
            internalLinks.add(l)
        elif parsed.netloc == internal_netloc:
             internalLinks.add(link.attrs['href']) 
    return list(internalLinks) 

In [ ]:
def getInternalLinks(bs, url): 
    netloc = urlparse(url).netloc 
    scheme = urlparse(url).scheme 
    internalLinks = set() 
    for link in bs.find_all('a'): 
        if not link.attrs.get('href'): 
            continue 
        parsed = urlparse(link.attrs['href']) 
        if parsed.netloc == '': 
            l = f'{scheme}://{netloc}/{link.attrs["href"].strip("/")}' 
            internalLinks.add(l) 
        elif parsed.netloc == netloc: 
            internalLinks.add(link.attrs['href']) 
    return list(internalLinks) 
def getExternalLinks(bs, url): 
    internal_netloc = urlparse(url).netloc 
    externalLinks = set() 
    for link in bs.find_all('a'): 
        if not link.attrs.get('href'): 
            continue 
        parsed = urlparse(link.attrs['href']) 
        if parsed.netloc != '' and parsed.netloc != internal_netloc: 
            externalLinks.add(link.attrs['href']) 
    return list(externalLinks) 

def getRandomExternalLink(startingPage): 
    bs = BeautifulSoup(urlopen(startingPage), 'html.parser') 
    externalLinks = getExternalLinks(bs, startingPage) 
    if not len(externalLinks): 
        print('No external links, looking around the site for one') 
        internalLinks = getInternalLinks(bs, startingPage) 
        return getRandomExternalLink(random.choice(internalLinks)) 
    else: 
        return random.choice(externalLinks)
    
def followExternalOnly(startingSite): 
    externalLink = getRandomExternalLink(startingSite) 
    print(f'Random external link is: {externalLink}') 
    followExternalOnly(externalLink) 

followExternalOnly('https://www.oreilly.com/')

In [ ]:
class Content:
    def __init__(self,url,title,body):
        self.url=url
        self.title=title.strip()
        self.body=body.strip()
    
    def print(self):
        print("Url:",self.url)
        print("Title:",self.title)
        print("Body:",self.body)
        
class Website:
    def __init__(self,name,url,titletag,bodytag):
        self.name=name
        self.url=url
        self.titletag=titletag
        self.bodytag=bodytag

class Crawler:
    def getPage(url):
        try:
            html = urlopen(url)
        except Exception:
            return None
        return BeautifulSoup(html, 'html.parser')
    
    def safeGet(bs, selector):
        selectedElems = bs.select(selector)
        if selectedElems is not None and len(selectedElems) > 0:
            return '\n'.join([elem.get_text() for elem in selectedElems])
        return ''
    
    def getContent(website, path):
        url = website.url + path
        bs = Crawler.getPage(url)
        if bs is not None:
            title = Crawler.safeGet(bs, website.titletag)
            body = Crawler.safeGet(bs, website.bodytag)
            return Content(url, title, body)
        return Content(url, '', '')   

  

    

In [ ]:
def scrapCNN(url):
    bs=BeautifulSoup(urlopen(url))
    title=bs.find('h1').text
    try:
        body=bs.find('div',{'class':'article__content'}).text
    except HTTPError:
        print("something of the error occured that is the http error")
    except URLError:
        print("URl error occured")
    except AttributeError:
        print("AttributeError occured")
    return Content(url,title,body)

content=scrapCNN("https://edition.cnn.com/2023/04/03/investing/dogecoin-elon-musk-twitter/index.html")  
content.print()  

In [ ]:
class Website:
    def __init__(self,url,name,targetpattern,absoluteurl,titleTag):
        self